In [21]:
import os

import pandas as pd
import numpy as np

import enlighten

from IPython.core.display import HTML
from IPython.display import SVG, Image, display

#RDKit related imports
from rdkit import RDLogger
from rdkit.Chem import PandasTools, AllChem as Chem, Descriptors
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator
from rdkit.Chem import Draw
from rdkit import DataStructs


In [22]:
def show(df):
    """Render the molecules within a DataFrame correctly"""
    return HTML(df.to_html(notebook=True))

In [23]:
dataset = 'harvard_hergcentral'

In [24]:
dir = f'{dataset}_OUTPUT'

In [5]:
try:
    os.mkdir(dir)    
except FileExistsError:
    print('Directory already exists')

Directory already exists


In [6]:
raw_path = '../DATASETS/herg_central.csv'

In [7]:
#load dataset

# hergcentral_raw = PandasTools.LoadSDF(raw_path, includeFingerprints=True)

hergcentral_raw = pd.read_csv('../DATASETS/herg_central.csv')

In [8]:
hergcentral_raw.head(5)

,ID,X,hERG_at_1uM,hERG_at_10uM,hERG_inhib
0,22416348,Cc1occc1C(=O)NCc1ccco1,20.17528,30.99165,0
1,26665387,COc1ccc(/C=C2\SC(=S)N(N3CCOCC3)C2=O)c(OC)c1,10.22630,13.05888,0
2,862531,C[C@H](NC(=O)Nc1cccc(C(F)(F)F)c1)C(=O)O,2.04420,0.06288,0
3,26732361,COc1cc(OC)c2ccc(=O)oc2c1C(CC(=O)N1CCOCC1)c1ccc...,21.80250,17.87858,0
4,49735227,COc1cccc(NC(=O)C2C3C=CC4(O3)C2C(=O)N(CCCN2CCCC...,8.33980,19.03128,0


In [9]:
# sid_list = hergcentral_raw['PubChem SID'].values
sid_list = hergcentral_raw['ID'].values

In [14]:
with open(f'./pubchem_convert/{dataset}_sidlist.txt', 'w') as cidfile:
    for i in sid_list:
        cidfile.write(str(i) + '\n')

## Convert the SIDs into CIDs, then use the CIDs to get the PubChem SDFs

In [15]:
# conversion list, need the reference between the SIDs and CIDs
match_case = r'^(\d*)\s*(\d*)$'

In [16]:
import re

In [17]:
SIDs_conv = []
CIDs_conv = []

In [18]:
with open(f'./pubchem_convert/{dataset}_sid2cid.txt') as conv_file:
    itera = conv_file.readlines()
    for i in itera:
        line_search = re.search(match_case, i, re.MULTILINE)
        SID = int(line_search.group(1))
        CID = int(line_search.group(2))
        SIDs_conv.append(SID)
        CIDs_conv.append(CID)

In [19]:
print(SIDs_conv[0])
print(CIDs_conv[0])

22416348
845080


In [20]:
def add_cid(row):
    try:
        sid = int(row['PubChem SID'])
        sid_index = SIDs_conv.index(sid)
        return CIDs_conv[sid_index]
    except Exception:
        return np.nan


In [21]:
def mol2smiles(row):
    try:
        mol = row.ROMol
        smiles = Chem.MolToSmiles(mol)
        return smiles
    except Exception:
        return np.nan

In [24]:
#for harvard hergcentral
hergcentral_raw = hergcentral_raw.rename(columns={'ID': 'PubChem SID'})
hergcentral_raw = hergcentral_raw.rename(columns={'X': 'SMILES'})


In [26]:
hergcentral_raw['PUBCHEM_CID'] = hergcentral_raw.apply(add_cid, axis=1)

In [27]:
show(hergcentral_raw.head(5))

,PubChem SID,SMILES,hERG_at_1uM,hERG_at_10uM,hERG_inhib,PUBCHEM_CID
0,22416348,Cc1occc1C(=O)NCc1ccco1,20.17528,30.99165,0,845080
1,26665387,COc1ccc(/C=C2\SC(=S)N(N3CCOCC3)C2=O)c(OC)c1,10.22630,13.05888,0,1945759
2,862531,C[C@H](NC(=O)Nc1cccc(C(F)(F)F)c1)C(=O)O,2.04420,0.06288,0,663760
3,26732361,COc1cc(OC)c2ccc(=O)oc2c1C(CC(=O)N1CCOCC1)c1ccc...,21.80250,17.87858,0,16746498
4,49735227,COc1cccc(NC(=O)C2C3C=CC4(O3)C2C(=O)N(CCCN2CCCC...,8.33980,19.03128,0,24793652


In [29]:
# padel_desc_file = pd.read_csv(f'{assay}_descriptors.csv')
padel_desc_file = pd.read_csv(f'./{dir}/{dataset}_descriptors.csv')

In [32]:
padel_desc_file = padel_desc_file.rename(columns={'Name': 'PUBCHEM_CID'})

In [33]:
padel_desc_file.head(5)

,PUBCHEM_CID,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,19,1,-0.1582,0.025027,40.1660,19.528758,0,0,17,11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,72,1,-0.1582,0.025027,40.1660,19.528758,0,0,17,11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6,0,2.1987,4.834282,42.1074,20.148379,0,0,16,13,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,86,1,-0.7918,0.626947,42.1103,20.493551,0,0,18,11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,109,2,-1.7727,3.142465,30.9194,17.155551,0,0,16,9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
comb_df = pd.merge(hergcentral_raw, padel_desc_file, on='PUBCHEM_CID')

In [35]:
show(comb_df.head(5))

In [36]:
comb_df = comb_df.drop_duplicates(subset=['PUBCHEM_CID'])

In [37]:
show(comb_df.head(5))

In [27]:
# comb_df = comb_df.drop(columns=['PUBCHEM_ASSAY_ID', 'PUBCHEM_SID', 'PUBCHEM_EXT_DATASOURCE_SMILES', 'PUBCHEM_ACTIVITY_SCORE'])

In [38]:
nan_cols = comb_df.isnull().sum(axis=0)
nan_cols.to_csv(f'{dir}/nancols.csv')

In [39]:
comb_df = comb_df[comb_df.columns[comb_df.isnull().sum() < 1000]]

In [40]:
nan_cols = comb_df.isnull().sum(axis=0)
nan_cols.to_csv(f'{dir}/nancols2.csv')

In [41]:
cmb_df_cp = comb_df.copy()

In [42]:
cmb_df_cp = cmb_df_cp.dropna()

In [43]:
nan_cols = cmb_df_cp.isnull().sum(axis=0)
nan_cols.to_csv(f'{dir}/nancols3.csv')

In [44]:
cmb_df_cp = cmb_df_cp.replace([np.inf, -np.inf], np.nan)

In [45]:
cmb_df_cp = cmb_df_cp.dropna()

In [46]:
ML_input_df = cmb_df_cp.copy()

In [ ]:
# mols = ML_input_df['ROMol'].to_list()

In [38]:
# smiles = []

# for i in mols:
#     smile = Chem.MolToSmiles(i)
#     smiles.append(smile)

In [39]:
# ML_input_df['SMILES'] = smiles

In [41]:
# smiles[0]

'N#Cc1cc2c(=O)n3ccccc3nc2n(Cc2ccco2)/c1=N/C(=O)c1ccco1'

In [48]:
show(ML_input_df.head(5))

In [42]:
# ML_input_df.drop(columns=['ROMol'])

,PubChem SID,hERG inhibition (%) at 1uM,hERG inhibition (%) at 10uM,ID,PUBCHEM_CID,nAcid,apol,naAromAtom,nAromBond,nAtom,...,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880,SMILES
0,22403712,-10.1124,-32.88202,atoms 31 bonds 35,1279881,0,56.096309,0,0,44,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N#Cc1cc2c(=O)n3ccccc3nc2n(Cc2ccco2)/c1=N/C(=O)...
1,22411435,5.5191,4.12648,atoms 29 bonds 31,4006429,0,57.947481,0,0,46,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,O=C(COC(=O)C1CC(O)CN1S(=O)(=O)c1ccc(Cl)cc1)c1c...
2,17513249,9.5532,5.23978,atoms 26 bonds 29,2000854,0,51.645481,0,0,43,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,O=C1/C(=C/c2ccccc2)Oc2cc(OC(=O)N3CCOCC3)ccc21
3,17512209,-0.211200000000005,-5.44332,atoms 25 bonds 27,6865519,0,47.821102,0,0,39,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cc1cc(C(=O)N/N=C/c2cccc([N+](=O)[O-])c2)c2cccc...
4,14729741,12.623,14.73378,atoms 36 bonds 40,1925701,0,72.187446,0,0,58,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,COCCNC(=O)c1c(NC(=O)c2ccc(Br)o2)n(Cc2ccccc2)c2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306888,57262925,4.85759999999999,1.38128,atoms 28 bonds 30,2553821,0,57.399067,0,0,47,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,O=C(COC(=O)c1ccc(S(=O)(=O)N2CCCC2)cc1)Nc1cccc(...
306889,57264745,4.4922,-3.78132,atoms 32 bonds 35,3568790,0,65.075032,0,0,56,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NC(=O)c1ccc(NC(=O)CN2CCN(S(=O)(=O)c3ccc4c(c3)O...
306890,57259638,-0.970100000000006,-13.56112,atoms 30 bonds 32,16303494,0,65.173411,0,0,57,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cc1cc(C)c(C)c(S(=O)(=O)N2CCN(C(=O)CNC(=O)c3ccc...
306891,57261004,-7.13740000000001,-25.02142,atoms 30 bonds 32,2429709,0,55.750067,0,0,49,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,O=C(COC(=O)c1ccc(O)cc1)Nc1cc(C(F)(F)F)ccc1N1CC...


In [49]:
ML_input_df.to_csv(f'{dir}/{dataset}_ML_raw_input.csv')

In [51]:
cmb_df_cp.to_csv(f'{dir}/{dataset}_treated_data.csv', index=False)

In [25]:
# start here if it all crashed lol
cmb_df_cp = pd.read_csv(f'{dir}/{dataset}_treated_data.csv')

In [26]:
show(cmb_df_cp.head(5))

In [27]:
cmb_label_df = cmb_df_cp[['PubChem SID', 'SMILES', 'hERG_at_1uM', 'hERG_at_10uM', 'PUBCHEM_CID']]

In [28]:
INHIB_LABELS = cmb_df_cp.hERG_inhib.to_numpy()

In [ ]:
# ACTIVITY_LABELS = cmb_df_cp['PUBCHEM_ACTIVITY_OUTCOME'].to_numpy()
# CID_markers = cmb_df_cp['PUBCHEM_CID'].to_numpy()

In [29]:
# cmb_df_cp = cmb_df_cp.drop(columns=['PubChem SID', 'SMILES', 'hERG_at_1uM', 'hERG_at_10uM', 'PUBCHEM_CID', 'hERG_inhib'])

#maybe keep the inhibition @ uM?
cmb_df_cp = cmb_df_cp.drop(columns=['PubChem SID', 'SMILES', 'PUBCHEM_CID', 'hERG_inhib'])

In [30]:
cmb_df_cp.head(5)

,hERG_at_1uM,hERG_at_10uM,nAcid,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,nH,nB,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,20.17528,30.99165,0,30.200723,0,0,26,15,11,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10.22630,13.05888,0,51.370274,0,0,42,24,18,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.04420,0.06288,1,32.971723,0,0,30,19,11,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,21.80250,17.87858,0,72.775790,0,0,64,34,30,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8.33980,19.03128,0,90.975306,0,0,82,40,42,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
from sklearn.feature_selection import VarianceThreshold

# selector = VarianceThreshold(threshold=(0.8 * (1 - 0.8)))
selector = VarianceThreshold()
VT_out = selector.fit_transform(cmb_df_cp)
selected_feature_names = selector.get_feature_names_out()


In [32]:
#use the VT features to remove these from the input
with open(f'{dir}/{dataset}_model_VT_features.txt', 'w') as vtfeat:
    for i in selected_feature_names:
        vtfeat.write(i + ',')

In [33]:
VT_out.shape

(298018, 1826)

In [34]:
VT_out[0]

array([20.17528, 30.99165,  0.     , ...,  0.     ,  0.     ,  0.     ])

In [35]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
Scaler_out = scaler.fit_transform(VT_out)

In [36]:
print(Scaler_out.shape)
print(INHIB_LABELS.shape)

(298018, 1826)
(298018,)


In [37]:
from sklearn.model_selection import train_test_split

In [40]:
x_train, x_test, y_train, y_test = train_test_split(Scaler_out, INHIB_LABELS, random_state=20)

In [ ]:
scaler_df = pd.DataFrame(Scaler_out)

In [ ]:
scaler_df.to_csv(f'{dir}/{dataset}_scaled_data.csv')

In [ ]:
np.savetxt(f'./{dir}/y_train.txt', y_train, fmt='%d')
np.savetxt(f'./{dir}/x_train.txt', x_train)
np.savetxt(f'./{dir}/y_test.txt', y_test, fmt='%d')
np.savetxt(f'./{dir}/x_test.txt', x_test)